In [ ]:
# SEED 설정
from functions import (
    set_seed,
    experiment1_hidden_size,
    experiment2_hidden_layers,
    experiment3_dropout,
    experiment4_dropout_with_layers
)

SEED = 42
set_seed(SEED)
print(f"random seed = {SEED}")

In [ ]:
# 데이터 로더 준비
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# MNIST 로드
mnist = load_dataset("mnist")

# Transform 정의
sample_data = torch.stack([
    transforms.ToTensor()(mnist['train'][i]['image'])
    for i in range(1000)
])
mean = sample_data.mean().item()
std = sample_data.std().item()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

def transform_dataset(dataset):
    def transform_fn(batch):
        images = [transform(img).view(-1) for img in batch["image"]]
        return {
            "image": torch.stack(images),
            "label": torch.tensor(batch["label"])
        }
    return dataset.with_transform(transform_fn)

train_dataset = transform_dataset(mnist["train"])
test_dataset = transform_dataset(mnist["test"])

batch_size = 128
test_batch_size = 1000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3
nb_epochs = 5

In [ ]:
# 실험 1


print("실험 1: Hidden Size 최적값 찾기 (50, 100~1000, 100개 단위로 증가하며 실험)")
hidden_sizes = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

results_exp1 = experiment1_hidden_size(
    hidden_sizes=hidden_sizes,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    learning_rate=learning_rate,
    nb_epochs=nb_epochs,
    seed=SEED
)

best_hidden_size = results_exp1['best_hidden_size']
best_acc = results_exp1['best_accuracy']

print(f"\n최고 성능: Hidden Size = {best_hidden_size}, 정확도 = {best_acc:.2f}%")

In [ ]:
# 실험 2


print(f"실험 2: Hidden Layers 개수 최적값 찾기 (1~10개)")
hidden_layers_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results_exp2 = experiment2_hidden_layers(
    hidden_layers_list=hidden_layers_list,
    best_hidden_size=best_hidden_size,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    learning_rate=learning_rate,
    nb_epochs=nb_epochs,
    seed=SEED
)

best_hidden_layers = results_exp2['best_hidden_layers']
best_acc_exp2 = results_exp2['best_accuracy']

print(f"\n최고 성능: Hidden Layers = {best_hidden_layers}개, 정확도 = {best_acc_exp2:.2f}%")

In [ ]:
# 실험 3

print(f"실험 3: Dropout 비율 최적값 찾기 (0.05~0.95 사이, 0.1 단위로 증가)")
dropout_rates = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95]

results_exp3 = experiment3_dropout(
    dropout_rates=dropout_rates,
    best_hidden_size=best_hidden_size,
    best_hidden_layers=best_hidden_layers,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    learning_rate=learning_rate,
    nb_epochs=nb_epochs,
    seed=SEED
)

best_dropout = results_exp3['best_dropout']
best_acc_exp3 = results_exp3['best_accuracy']
print(f"\n최고 성능: Dropout = {best_dropout}, 정확도 = {best_acc_exp3:.2f}%")

In [ ]:
# 실험 4


print(f"실험 4: Dropout 여부에 따른 차이 비교\nHidden Layers 1~10개에 Dropout를 적용했을 때 실험2와의 차이를 비교합니다.")
hidden_layers_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results_exp4 = experiment4_dropout_with_layers(
    hidden_layers_list=hidden_layers_list,
    best_hidden_size=best_hidden_size,
    best_dropout=best_dropout,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    learning_rate=learning_rate,
    nb_epochs=nb_epochs,
    seed=SEED
)

best_hidden_layers_exp4 = results_exp4['best_hidden_layers']
best_acc_exp4 = results_exp4['best_accuracy']

print(f"\n최고 성능: Hidden Layers = {best_hidden_layers_exp4}개, 정확도 = {best_acc_exp4:.2f}%")

In [ ]:
# 다른 파일에서 시각화 하기 위해 결과 저장

'''이 셀은 ai로 작성했습니다.'''

import pickle

all_results = {
    'experiment1': results_exp1,
    'experiment2': results_exp2,
    'experiment3': results_exp3,
    'experiment4': results_exp4,
    'seed': SEED,
    'best_hidden_size': best_hidden_size,
    'best_hidden_layers': best_hidden_layers,
    'best_dropout': best_dropout,
    'best_hidden_layers_exp4': best_hidden_layers_exp4
}

with open('experiment_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print("모든 실험 결과가 'experiment_results.pkl'에 저장되었습니다.")
print(f"\n최종 최적 하이퍼파라미터:")
print(f"  - Hidden Size: {best_hidden_size}")
print(f"  - Hidden Layers (Dropout 없음): {best_hidden_layers}")
print(f"  - Dropout 비율: {best_dropout}")
print(f"  - Hidden Layers (Dropout 있음): {best_hidden_layers_exp4}")